Воспользуемся сверточной нейросетью на примере задачи определения частей речи слов "POS-tagging" (part of speech tagging). pyconll — потребуется для загрузки корпуса. В этом семинаре мы будем использовать размеченный корпус, который называется "SynTag Rus". Этот корпус был размечен руками, лингвистами, и в нём содержится разметка по частям речи, по нормальным формам слов, синтаксическая разметка. Он предназначен для того, чтобы настраивать и проверять методы лингвистического анализа текстов, а именно — морфологического разбора и синтаксического разбора. Разметка в этом корпусе представлена в формате CoNLL — это достаточно распространённый формат для того, чтобы хранить аннотированные деревья и разную лингвистическую разметку.  

# Свёрточные нейросети и POS-теггинг

POS-теггинг - определение частей речи (снятие частеречной неоднозначности)

In [1]:
# Если Вы запускаете ноутбук на colab или kaggle,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

# !git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
# import sys; sys.path.append('./stepik-dl-nlp')
# !pip install pyconll

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

import dlnlputils
from dlnlputils.data import tokenize_corpus, build_vocabulary, \
    character_tokenize, pos_corpus_to_tensor, POSTagger
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()

## Загрузка текстов и разбиение на обучающую и тестовую подвыборки

In [2]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# !wget -O ./stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/r2.8/ru_syntagrus-ud-train.conllu
# !wget -O ./stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/r2.8/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/r2.8/ru_syntagrus-ud-dev.conllu

# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
full_train = pyconll.load_from_file('./datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./datasets/ru_syntagrus-ud-dev.conllu')

--2022-02-08 23:51:57--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/r2.8/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M  33.0MB/s    in 2.3s    

2022-02-08 23:52:02 (33.0 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2022-02-08 23:52:03--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/r2.8/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent

Возьмём пару первых предложений и посмотрим, как выглядит разметка для задачи определения частей речи слов. Предложения разделены пустой строкой, то есть первое предложение — это, по сути, просто заголовок: "анкета", "точка". И мы видим, что "анкета" — это существительное, а "точка" получила тэг "PUNCT", то есть — пунктуация. Второе предложение гораздо длиннее, и в нём для каждого токена проставлены тэги частей речи. 

In [3]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



Посчитаем пару статистик по нашему корпусу. В первую очередь нас интересует наибольшая длина предложения и наибольшая длина токена. Выведем несколько первых предложений. Решать задачу определения части речи мы будем с помощью свёрточных нейросетей, причём будем использовать нейросети, которые принимают на вход номера отдельных символов — то есть они работают не на уровне целых токенов а на уровне символов. Это вполне оправдано, потому что часть речи во многом определяется структурой слова, наличием суффиксов, окончаний определённого вида. Если бы мы работали на уровне отдельных токенов, то мы бы просто не могли анализировать структуру слов, нам бы приходилось просто запоминать, что такое-то слово — это существительное, другое слово это просто глагол. Следующий этап обработки корпуса — это построение словаря символов.

In [4]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [5]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

Словарь символов — это стандартный "dict", который отображает подстроку, содержащую единственный элемент (это сам символ) в номер этого символа. Можем каждый токен представить как список чисел. Также нам опять пригодится фиктивный символ, который будет означать отсутствие символа. Он будет использоваться для того, чтобы уравнять длины всех токенов и всех предложений. Как мы видим, всего у нас в корпусе 150 уникальных символов. Самый частотный символ — это пробел, затем идут гласные. Аналогичную процедуру проделаем и с метками частей речи. Для того, чтобы составить обучающую выборку, нам нужно преобразовать строковые метки частей речи в их номера. Как мы видим, всего в корпусе 17 частей речи и мы добавляем сюда фиктивную "часть речи" — опять же, для того, чтобы выравнивать длины предложений. Что же значат эти метки частей речи? "VERB" — это глагол, "PROPN" — это имя собственное, "NUM" — это числительное, "NOUN" — это существительное. Ну, и наконец, мы перекладываем весь исходный корпус в специальные структуры, для того, чтобы подавать их в наш цикл для обучения. То есть мы перекладываем их в pytorch Dataset. Мы используем стандартный TensorDataset, он принимает на вход список тензоров, то есть, для того чтобы использовать этот Dataset, нам нужно подготовить эти тензоры. Первый — это тензор идентификаторов символов, а второй — это идентификаторы меток частей речи. Рассмотрим поподробнее, как именно мы составляем эти тензоры. Функция, которая нас интересует — на вход получает список токенизированных предложений. На самом деле, это не просто список токенизированных предложений, а это результаты разбора исходного корпуса в формате CoNLL, то есть в нём токены имеют дополнительную информацию, а именно — информацию о частях речи. То есть эта функция предназначена для преобразования обучающей и тестовой выборки в 2 тензора тензора — тензор входных идентификаторов символов и тензор выходных идентификаторов тэгов. Кроме списка исходных предложений нам нужно отображение из символов в номера символов, а также отображение из меток в номера; нам нужна оценка максимальной длины предложения и максимальной длины токена в символах. Создаём заготовки для тензоров — сначала мы эти тензоры инициализируем нулями. Тензор номеров символов имеет следующую размерность: \[количество предложений (всего в обучающей выборке) на количество токенов в предложений на максимальную длину токена]. Но, кроме этого, мы добавляем ещё 2 дополнительных колоночки к каждому токену. Эти колоночки дополнительные мы будем заполнять нулями — они нам нужны для того, чтобы указать нейросети, что определённая N-грамма символов встречается именно в начале токена или именно в конце токена, но не в середине. То есть это нужно для того, чтобы нейросеть умела отличать начало слова и конец слова от середины слова.

In [6]:
# каждый токен можем представить как список чисел
train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

Количество уникальных символов 150
[('<PAD>', 0), (' ', 1), ('о', 2), ('е', 3), ('а', 4), ('т', 5), ('и', 6), ('н', 7), ('.', 8), ('с', 9)]


In [7]:
# Аналогично посчитаем для частей речи
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

Процедура заполнения тензоров достаточно простая — итерируемся по всем предложениям, по всем токенам в предложении, и для каждого токена кладём в соответствующую ячейку тензора меток идентификатор метки, а также кладём в соответствующую ячейку входного тензора идентификатор очередного символа. Теперь посмотрим, как данные будут выглядеть для нейросети во время обучения. Для этого выведем фрагмент тензора, представляющего второе предложение из обучающей выборки — первые пять слов этого предложения. Мы видим прямоугольный тензор, каждая строчка этого тензора представляет один токен и содержит номера символов, которые в этом токене используются, в том порядке, в котором они встречались в самом токене. Обратить внимание здесь нужно на две вещи — первая: это стартовый нолик, он нам нужен для того, чтобы указать нейросети, что это — начало токена. Всё, что после последнего символа — заполняется нулями. Количество значимых элементов в каждой такой строчке у нас отличается. У нас есть короткие и длинные токены. Так выглядит входной тензор. Давайте теперь посмотрим, как выглядит тензор меток. Целевой тензор для этого же предложения — мы выбрали второе предложение из обучающей выборки, и это уже не двухмерный тензор, это одномерный тензор, то есть это просто список чисел, каждое число представляет номер тэга соответствующего токена. Для первого токена из этого предложения мы должны будем предсказать класс номер "8", для 2 — класс номер "1". Для токенов фиктивных, ненастоящих, мы всегда должны будем предсказывать "0". Фиктивные токены у нас не являются частью исходного предложения, они используют для того, чтобы выровнять длины всех предложений и иметь возможность упаковывать предложения разной длины в прямоугольный тензор и обрабатывать в пакетном режиме в нейросети. Это нужно для того, чтобы эффективно использовать возможности современных вычислителей — видеокарт. Определим вспомогательный нейросетевой модуль. Он состоит из свёрток, функции активации и дропаута. Посмотрим на функцию "forward". В основе модуля лежит набор одинаковых блоков. Попробуем изобразить этот модуль графически. У нас в начале есть "x", он подаётся в некий блок "layer 1", затем выход "layer 1" приплюсовывается к его же входу. Это всё подаётся в следующий — такой же блок "layer 2" и снова приплюсовывается. Это простенький ResNet. Использование вот этих связей (skip connection) ускоряет сходимость, а также позволяет нам сделать нейросеть более глубокой. Другими словами, без "skip connection" мы можем сделать нейросеть максимум из 5...9 блоков глубиной, но, используя "skip connection", мы можем делать нейросеть произвольной глубины, она по-прежнему будет обучаться. Давайте теперь посмотрим, как же устроен каждый из вот этих блоков. Каждый из этих блоков реализуется с помощью модуля pytorch "sequential" — это базовый модуль, который берёт список других модулей и выполняет их по-очереди, передавая результат первого во второй, из второго в третий, и так далее. В этом блоке первый слой — это свёрточный слой. Это одномерные свёртки — для текстов чаще всего используются одномерные свёртки. По умолчанию, мы говорим, что размер ядра равен 3. При этом, свёрточный слой не меняет количества каналов — он принимает одно и то же число каналов и возвращает одно и то же число каналов. Кроме того, здесь мы используем padding. Мы делаем, чтобы размерность тензора вообще никак не менялась, то есть по умолчанию при нулевом padding свёртки немного сжимают тензор по пространственному измерению. Здесь, чтобы размерность тензора оставалась прежней. Для этого, перед тем, как применять свёртки, нужно добавить какое-то количество нулей в начало и в конец тензора. Реализацию свёрточного модуля мы рассмотрим попозже. Второй слой в нашем блоке — это dropout. Он нужен для того, чтобы нейросеть меньше переобучалась. В режиме обучения dropout зануляет случайные ячейки тензора. Когда нейросеть обучена, dropout ничего не делает. Третий слой блока — это функция активации. Здесь мы используем Leaky ReLU, часто это — неплохой выбор. Таким образом, мы определили достаточно универсальный свёрточный модуль, который можно использовать в абсолютно разных ситуациях и, в рамках этого семинара, мы будем использовать его, как минимум, в двух разных случаях. !

In [8]:
# перекладываем их в pytorch Dataset. Мы используем стандартный TensorDataset, он 
# принимает на вход список тензоров, то есть, для того чтобы использовать этот 
# Dataset, нам нужно подготовить эти тензоры. 

train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
test_dataset = TensorDataset(test_inputs, test_labels)
# из исходников
# def pos_corpus_to_tensor(sentences, char2id, label2id, max_sent_len, max_token_len):
#     inputs = torch.zeros((len(sentences), max_sent_len, max_token_len + 2), dtype=torch.long)
#     targets = torch.zeros((len(sentences), max_sent_len), dtype=torch.long)

#     for sent_i, sent in enumerate(sentences): Итерируемся по предложениям
#         for token_i, token in enumerate(sent): итерируемся по токенам
#             targets[sent_i, token_i] = label2id.get(token.upos, 0) кладем в нужное место токен
#             for char_i, char in enumerate(token.form):
#                 inputs[sent_i, token_i, char_i + 1] = char2id.get(char, 0) идентификатор очередного символа

#     return inputs, targets

pos_corpus_to_tensor на вход получает список токенизированных предложений. На самом деле, это не просто список токенизированных предложений, а это результаты разбора исходного корпуса в формате CoNLL, то есть в нём токены имеют дополнительную информацию, а именно — информацию о частях речи. То есть эта функция предназначена для преобразования обучающей и тестовой выборки в 2 тензора тензора — тензор входных идентификаторов символов и тензор выходных идентификаторов тэгов. Кроме списка исходных предложений нам нужно отображение из символов в номера символов, а также отображение из меток в номера; нам нужна оценка максимальной длины предложения и максимальной длины токена в символах. Создаём заготовки для тензоров — сначала мы эти тензоры инициализируем нулями. Тензор номеров символов имеет следующую размерность: [количество предложений (всего в обучающей выборке) на количество токенов в предложений на максимальную длину токена]. Но, кроме этого, мы добавляем ещё 2 дополнительных колоночки к каждому токену. Эти колоночки дополнительные мы будем заполнять нулями — они нам нужны для того, чтобы указать нейросети, что определённая N-грамма символов встречается именно в начале токена или именно в конце токена, но не в середине. То есть это нужно для того, чтобы нейросеть умела отличать начало слова и конец слова от середины слова.

In [9]:
train_inputs[1][:5] # первые 5 слов второго предложения
# получаем номера символов которые в этом токене используются
# стартовый нолик, он нам нужен для того, чтобы указать нейросети, что это — 
# начало токена. Всё, что после последнего символа — заполняется нулями. 
# Количество значимых элементов в каждой такой строчке у нас отличается. 
# У нас есть короткие и длинные токены

tensor([[ 0, 39,  4, 25,  4, 11, 20,  7,  6, 13,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  2, 23, 11,  4,  9,  5,  7,  2, 22,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 17, 16, 10,  4, 12, 11,  3,  7,  6, 19,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  9, 12, 19, 21,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 40,  3, 15,  3,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0

In [10]:
train_labels[1]
# Целевой тензор для этого же предложения — мы выбрали второе предложение из 
# обучающей выборки, и это уже не двухмерный тензор, это одномерный тензор, то 
# есть это просто список чисел, каждое число представляет номер тэга 
# соответствующего токена. Для первого токена из этого предложения мы должны будем
# предсказать класс номер "8", для 2 — класс номер "1". Для токенов фиктивных, 
# ненастоящих, мы всегда должны будем предсказывать "0". Напомню, что фиктивные 
# токены у нас не являются частью исходного предложения, они используют для того,
# чтобы выровнять длины всех предложений и иметь возможность упаковывать 
# предложения разной длины в прямоугольный тензор и обрабатывать в пакетном 
# режиме в нейросети. Это нужно для того, чтобы эффективно использовать 
# возможности современных вычислителей — видеокарт.

tensor([ 8,  1,  8,  8, 12, 12,  4,  8,  1, 13, 16,  2,  8,  3,  3, 13, 16,  2,
         8,  2,  8,  5,  3, 10, 16,  2,  8,  8,  2,  8, 13,  8, 13, 13,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0])

## Вспомогательная свёрточная архитектура

In [11]:
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for _ in range(layers_n):
            layers.append(nn.Sequential(
                # Одномерная свертка. Ядро равно 3. Паддинг чтобы размерность не менялась
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                nn.Dropout(dropout), # для меньшего переобучения 
                nn.LeakyReLU())) # активация
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers: # каждый layer это sequential
            x = x + layer(x) # выход с layer1 суммируется с исходным сигналом и подается в layer2
            # это простой resnet
        return x

Первая попытка решить нашу задачу. Для этого нам необходимо описать нейросетевой модуль для pytorch. Он будет предсказывать метки частей речи токенов, используя информацию, содержащуюся только в самих токенах. Другими словами, эта модель никак не будет использовать контекст, в котором слово употребляется. Опять же, давайте начнём с метода "forward". Для начала, получим переменные, представляющие форму исходного тензора. На вход нам приходит трёхмерный тензор, размерности которого соответствуют \[размеру батча, наибольшей длине предложения в токенах и наибольшей длине каждого токена в символах]. Далее мы схлопываем первое и второе измерения, чтобы получить двухмерный тензор. Другими словами, мы забываем о том, что токены у нас были как-то объединены в предложения (для этой модели нам совершенно неважно). Далее мы используем embedding-слой для того, чтобы для каждого символа получить вектора. Таким образом, мы снова получаем трёхмерный тензор, который соответствует \[количеству токенов в батче на длину каждого токена на размер вектора для символа]. Затем мы транспонируем этот тензор для того, чтобы мы могли его подать в свёрточную нейросеть. В pytotch принята следующая конвенция о порядке размерностей: сначала идёт размер батча, затем идёт количество признаков для каждого элемента (в данном случае — это размер вектора), а затем идёт какое-то количество размерностей, соответствующих самим элементам, то есть дальше идут пространственные измерения. В текстах мы работаем с одномерными данными, то есть у нас, в данном случае это длина токена. После всех этих операций, переменная "char embeddings" содержит векторы для отдельных символов. Пока что, эти векторы не содержат информации о том, в каком контексте используется каждый символ — это просто какие-то априорные знания о том, что это вообще за символ. Вначале обучения эти вектора инициализируется случайными числами. Затем эти вектора мы передаём в свёрточный модуль, для того, чтобы учесть локальный контекст — то есть ту ситуацию, в которой каждый конкретный символ используется. Для этого мы применяем "backbone"-сетку, роль которой выполняет простенький ResNet, который мы определили чуть выше. В результате мы получаем трёхмерный тензор такой же размерности кладём его в переменную features, и эта переменная содержит векторы символов уже с учётом к их контекста. Тут нужно вспомнить, что тэги нам нужно предсказывать не для каждого символа, а для каждого токена. Поэтому нам нужно как-то агрегировать признаки символов в токене, чтобы получить вектор токена. Для этого мы используем pooling — в данном случае это max pooling. Допустим, у нас есть некоторая матричка, строки в этой матричке представляют отдельные символы в токене, а столбцы — это признаки этих символов. И max pooling делает из этого один вектор — количество элементов в этом векторе соответствует количеству столбцов в исходной матрице, и каждый элемент получен взятием функции максимума из соответствующего столбца. В результате применения global пулинга мы получаем тензор, на этот раз — уже двумерный, каждая строчка этого тензора представляет отдельный токен. Ну и наконец, по каждому токену нам нужно принять решение касательно его метки. Для этого признаки токенов мы передаём в ещё один нейросетевой модуль, который называется out — это просто полносвязный блок. В результате применения этого блока мы получаем также двухмерный тензор, но у него уже размер строки не "embedding size", a "количество меток частей речи". Далее мы меняем форму этого тензора, преобразуем его в трёхмерный, то есть "вспоминаем" о том, что у нас есть предложения, и транспонируем для того, чтобы порядок измерений соответствовал порядку измерений в исходном тензоре "tokens". В чём же физический смысл того, что мы только что рассмотрели? Физический смысл того, что мы сейчас описали, заключается в том, чтобы рассмотреть все возможные N-граммы символов, которые встречаются в каждом токене, и по ним попробовать определить часть речи. Благодаря тому, что основная наша нейросеть (backbone) содержит "skip connections", N-граммы, которые учитываются этой нейросетью, по сути, имеют различную длину. Например, если мы используем размер ядра свёртки, равный 3, то первый блок учитывает трёхграммы, второй блок уже учитывает пятиграммы, а третий — семиграммы, соответственно. При этом, благодаря тому, что есть "skip connection", информация о трёхграммах не теряется, она пробрасывать до самого конца.


## Предсказание частей речи на уровне отдельных токенов

In [12]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape # формы тензоров
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len) # схлопываем тензоры
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        # После всех этих операций, переменная "char embeddings" содержит 
        # векторы для отдельных символов. Пока что, эти векторы не содержат 
        # информации о том, в каком контексте используется каждый символ — это 
        # просто какие-то априорные знания о том, что это вообще за символ. 
        # Вначале обучения эти вектора инициализируется случайными числами. 
        # Затем эти вектора мы передаём в свёрточный модуль, для того, чтобы 
        # учесть локальный контекст — то есть ту ситуацию, в которой каждый 
        # конкретный символ используется. Для этого мы применяем "backbone"-сетку,
        #  роль которой выполняет простенький ResNet, который мы определили выше.
        features = self.backbone(char_embeddings) # В результате мы получаем трёхмерный тензор такой же размерности 
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

Физический смысл того, что мы сейчас описали, заключается в том, чтобы рассмотреть все возможные N-граммы символов, которые встречаются в каждом токене, и по ним попробовать определить часть речи. Благодаря тому, что основная наша нейросеть (backbone) содержит "skip connections", N-граммы, которые учитываются этой нейросетью, по сути, имеют различную длину. Например, если мы используем размер ядра свёртки, равный 3, то первый блок учитывает трёхграммы, второй блок уже учитывает пятиграммы, а третий — семиграммы, соответственно. При этом, благодаря тому, что есть "skip connection", информация о трёхграммах не теряется, она пробрасывать до самого конца.

Попробуем обучить эту нейросеть и посмотрим, как хорошо мы можем определять часть речи слова, не используя информацию о его контексте. Сначала мы создаём экземпляр описанного нами нейросетевого модуля, передаём ему в конструктор количество уникальных символов в датасете, количество меток, рабочий размер модели, то есть каждый символ мы будем представлять вектором из 64 элементов, дальше мы указываем количество свёрточных блоков, то есть глубину нашей нейросети (мы будем использовать глубину равную трём), размер ядра свёртки, а также вероятность дропаута. Вероятность равная 0.3 означает, что на каждом проходе по нейросети в режиме обучения будет, случайным образом, зануляться примерно треть активации.  

In [13]:
single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), embedding_size=64, layers_n=3, kernel_size=3, dropout=0.3)
print('Количество параметров', sum(np.product(t.shape) for t in single_token_model.parameters()))

Количество параметров 47826


Далее мы используем нашу стандартную функцию для тренировки нейросетей, которая описана в нашей библиотеке специально для этого курса, передаём туда модель, датасеты, а также указываем функцию потерь — мы будем использовать кросс-энтропию, эта функция потерь используется в многоклассовых задачах. Полное обучение описанной нейросети до сходимости на этом корпусе требует примерно одного часа работы видеокарты. Мы не будем ждать всё это время — мы просто загрузим модельку, которую я обучил заранее и оценим её качество. .

In [14]:
(best_val_loss,
 best_single_token_model) = train_eval_loop(single_token_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=10,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=500,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                       factor=0.5,
                                                                                                                       verbose=True))

Эпоха 0
Эпоха: 501 итераций, 45.69 сек
Среднее значение функции потерь на обучении 0.07997052162558495
Среднее значение функции потерь на валидации 0.03771348909871413
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 44.95 сек
Среднее значение функции потерь на обучении 0.02819045022605898
Среднее значение функции потерь на валидации 0.02991329000727965
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 44.95 сек
Среднее значение функции потерь на обучении 0.024262320893461593
Среднее значение функции потерь на валидации 0.02679025984857932
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 44.93 сек
Среднее значение функции потерь на обучении 0.022401973770600474
Среднее значение функции потерь на валидации 0.0258692220351336
Новая лучшая модель!

Эпоха 4
Эпоха: 501 итераций, 44.95 сек
Среднее значение функции потерь на обучении 0.021007792802969973
Среднее значение функции потерь на валидации 0.02294884014432088
Новая лучшая модель!

Эпоха 5
Эпоха: 501 итераций, 44.94 сек
Среднее з

In [15]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_single_token_model.state_dict(), './models/3.7.single_token_pos.pth')

In [16]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
single_token_model.load_state_dict(torch.load('./models/3.7.single_token_pos.pth'))

<All keys matched successfully>

Для того, чтобы оценить качество, мы используем обученную нейросеть, чтобы получить предсказание для обучающей выборки, а дальше используем функцию "classification_report" из библиотеки scikit-learn. Эта функция выдаёт самые стандартные метрики качества классификации (а именно, точность, полнота и f-мера) для каждого класса. А также, для каждого класса выдаётся количество примеров. Как мы видим, в этом случае датасет имеет сильно скошенное распределение классов, другими словами у нас есть очень частые классы и таких классов мало, а есть очень редкие классы. В таком случае нам вообще нет смысла использовать "accuracy", то есть долю верно угаданных ответов — она абсолютно неинформативна. На обучающей выборке она равна единице, хотя есть классы, которые не очень хорошо определяются. В случае сильно скошенного распределения классов, самое правильное — это считать сразу несколько метрик, устойчивых к распределению классов (в данном случае, это — точность, полнота и f-мера) и смотреть на них всех. Часто бывает удобно получить не большую пачку цифр, а всего лишь одно число, по которому мы сможем понять, насколько хорошо модель работает. В данном случае лучше всего нам подходит "macro-среднее". Что значит "macro-среднее"? Это значит, что сначала мы считаем каждую метрику по каждому классу, а потом усредняем. Macro-среднее более устойчиво к скошенным распределениям классов

In [17]:
train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

1526it [00:28, 54.04it/s]                               


Среднее значение функции потерь на обучении 0.018453510478138924
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.83      0.96      0.89     85589
         ADP       1.00      0.99      0.99     81963
         ADV       0.93      0.82      0.87     44101
         AUX       0.87      0.63      0.73      7535
       CCONJ       0.88      0.98      0.93     30432
         DET       0.90      0.75      0.82     21968
        INTJ       0.58      0.09      0.16        78
        NOUN       0.98      0.92      0.95    214497
         NUM       0.96      0.94      0.95     13746
        PART       0.96      0.78      0.86     26638
        PRON       0.86      0.89      0.87     38438
       PROPN       0.80      0.95      0.87     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.80      0.93      0.86     16219
         SYM       1.00      0.99      1.00       840
        VERB    

100%|██████████| 206/205.75 [00:03<00:00, 55.13it/s]


Среднее значение функции потерь на валидации 0.019972970709204674
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.82      0.95      0.88     11222
         ADP       0.99      0.99      0.99     10585
         ADV       0.92      0.80      0.86      6165
         AUX       0.87      0.56      0.68      1108
       CCONJ       0.89      0.98      0.93      4410
         DET       0.89      0.72      0.80      3085
        INTJ       0.00      0.00      0.00        11
        NOUN       0.97      0.92      0.95     27974
         NUM       0.95      0.92      0.93      1829
        PART       0.96      0.79      0.87      3875
        PRON       0.85      0.88      0.86      5598
       PROPN       0.79      0.94      0.86      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.78      0.94      0.85      2258
         SYM       1.00      0.96      0.98        53
        VERB   

Мы видим, что, несмотря на то, что модель была очень простая и она никак не учитывала контексты, на обучающей выборке она показывает достаточно высокую f-меру (0.9). На валидационной выборке macro-средняя f-мера не сильно меньше, всего лишь на 1% (0.89 — достаточно неплохо).   




## Предсказание частей речи на уровне предложений (с учётом контекста)

Частиречная омонимия — это когда слова "с разной частью речи" пишутся абсолютно одинаково. Например, "мама мыла раму": здесь "мыла" — это глагол. Или "в ванной не было мыла": здесь "мыла" — это уже существительное. Такие случаи принципиально невозможно отлавливать, используя только информацию изнутри слова — нам нужно учитывать контекст в предложении. Давайте опишем вторую модельку, которая учитывает такой контекст. Опять же, давайте посмотрим на метод forward. Первая часть этого метода — ровно такая же, как и в предыдущей модели. Берём номера символов, делаем выборку из таблицы эмбеддингов, получаем векторы символов. Потом, с помощью свёрточной нейросети, которая в данном случае называется "single token backbone", получаем векторы символов с учётом их контекста. Затем используем глобальный пулинг для того, чтобы получить вектор токена. Далее мы проделываем примерно то же самое, но уже на уровне токенов в предложении. Сначала мы немного изменяем форму и транспонируем тензор признаков таким образом, чтобы получить трёхмерный тензор с размерностями "количество предложений в батче", "количество признаков для каждого токена" и "количество токенов в предложении". Этот тензор содержит признаки токенов без учёта их контекста. Далее мы эти признаки подаём в другой свёрточный модуль для того, чтобы учесть это самый контекст. Для учёта контекста символов и для учёта контекста слов мы используем две разных нейросети, но архитектура их одинакова — это ровно та самая свёрточная сеть со "skip connections", которую мы описали в начале нашего семинара. Несмотря на то, что архитектура у этих модулей одинаковая, веса у них отличаются. На старте обучения они инициализируются случайно, а затем — настраиваются. В результате мы получаем также трёхмерный тензор, той же размерности, но теперь — вектор для каждого токена уже содержит информацию о его контексте. И, наконец, нам нужно принять решение о части речи каждого токена. Для этого мы проецируем признаки каждого токена в пространство классов. Для этого мы используем одномерный свёрточный блок с ядром свёртки 1, то есть он за раз видит только один токен. Физический смысл того, что мы описали сейчас, заключается в том, чтобы сначала проанализировать структуру каждого слова, найти там какие-то суффиксы и окончания (за это отвечает первая часть) а затем — смешать информацию о структуре каждого слова с контекстом, в котором это слово употребляется.

In [18]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

Сначала мы создаём экземпляр класса, который только что описали, и передаём туда, на самом деле, все те же самые параметры — это количество символов, количество выходных меток, количество признаков для каждого символа, но теперь это ещё и количество признаков для каждого токена (оно у нас одинаково), а также два набора параметров для свёрточных модулей. Первый набор параметров — для анализа символов на уровне каждого токена, а второй набор параметров — для анализа контекста токенов. Мы решили задать одни и те же параметры, потому что это неплохо работает. В результате мы получили нейросеть, в которой в два раза больше параметров

In [19]:
sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
                                              context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 84882


Далее мы используем всё тот же цикл обучения — здесь никаких существенных отличий нет. Если вы потом внимательно посмотрите на вывод обучения, вы увидите, что для второй модели, которая учитывает контекст токенов, значение функции потерь во время обучения падает гораздо быстрее и, в итоге, достигает существенно меньшего значения. е

In [20]:
(best_val_loss,
 best_sentence_level_model) = train_eval_loop(sentence_level_model,
                                              train_dataset,
                                              test_dataset,
                                              F.cross_entropy,
                                              lr=5e-3,
                                              epoch_n=10,
                                              batch_size=64,
                                              device='cuda',
                                              early_stopping_patience=5,
                                              max_batches_per_epoch_train=500,
                                              max_batches_per_epoch_val=100,
                                              lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                         factor=0.5,
                                                                                                                         verbose=True))

Эпоха 0
Эпоха: 501 итераций, 45.92 сек
Среднее значение функции потерь на обучении 0.06945245525615658
Среднее значение функции потерь на валидации 0.026779627989940713
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 45.93 сек
Среднее значение функции потерь на обучении 0.02525442752786382
Среднее значение функции потерь на валидации 0.019473401589853927
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 45.93 сек
Среднее значение функции потерь на обучении 0.020784088282171122
Среднее значение функции потерь на валидации 0.0157019424224549
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 45.95 сек
Среднее значение функции потерь на обучении 0.018439713393351036
Среднее значение функции потерь на валидации 0.014976288957318456
Новая лучшая модель!

Эпоха 4
Эпоха: 501 итераций, 45.96 сек
Среднее значение функции потерь на обучении 0.01716930501742991
Среднее значение функции потерь на валидации 0.013994390827299344
Новая лучшая модель!

Эпоха 5
Эпоха: 501 итераций, 45.94 сек
Средне

In [21]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_sentence_level_model.state_dict(), './models/3.7.sentence_level_pos.pth')

In [22]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
sentence_level_model.load_state_dict(torch.load('./models/3.7.sentence_level_pos.pth'))

<All keys matched successfully>

Здесь у нас macro f-мера, равна 0.93, а для первой модели, как мы помним, она была равна 0.89. То есть, мы получили прирост в 4%, добавив всего лишь ещё один свёрточный модуль для учёта контекста токенов.

In [23]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

1526it [00:27, 54.60it/s]                               


Среднее значение функции потерь на обучении 0.010619516484439373
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.91      0.95      0.93     85589
         ADP       1.00      0.99      0.99     81963
         ADV       0.89      0.93      0.91     44101
         AUX       0.92      0.88      0.90      7535
       CCONJ       0.93      0.99      0.96     30432
         DET       0.85      0.95      0.90     21968
        INTJ       1.00      0.15      0.27        78
        NOUN       0.98      0.96      0.97    214497
         NUM       0.95      0.96      0.95     13746
        PART       0.97      0.86      0.91     26638
        PRON       0.98      0.88      0.93     38438
       PROPN       0.90      0.98      0.94     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.91      0.84      0.87     16219
         SYM       1.00      1.00      1.00       840
        VERB    

100%|██████████| 206/205.75 [00:03<00:00, 54.67it/s]


Среднее значение функции потерь на валидации 0.011860478669404984
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.91      0.94      0.92     11222
         ADP       1.00      0.99      0.99     10585
         ADV       0.89      0.92      0.90      6165
         AUX       0.90      0.88      0.89      1108
       CCONJ       0.93      0.99      0.96      4410
         DET       0.83      0.94      0.88      3085
        INTJ       1.00      0.27      0.43        11
        NOUN       0.97      0.95      0.96     27974
         NUM       0.94      0.95      0.94      1829
        PART       0.97      0.86      0.91      3875
        PRON       0.97      0.88      0.92      5598
       PROPN       0.88      0.97      0.93      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.89      0.84      0.86      2258
         SYM       0.98      0.98      0.98        53
        VERB   

## Применение полученных теггеров и сравнение

Для того, чтобы вручную поэкспериментировать с обученными моделями мы сделали специальный класс "POS tagger", который принимает на вход а обученную модель, а также те же самые параметры — это отображение символов в их идентификаторы, это количество уникальных тэгов. Кроме этого, он на вход принимает отображение символов в их номера, отображение из номеров тэгов обратно в их строковое представление, а также статистики корпуса — максимальную длину предложения и максимальную длину токена. И мы создаём два экземпляра этого класса для модели, обученной на отдельных токенах, и для модели, которая учитывает контекст. Класс "POS tagger" достаточно простой. В конструктор он принимает все эти параметры, которые мы сейчас описали, и у него есть ещё метод — этот метод применяет нашу обученную модель к переданным предложениям для того, чтобы получить части речи токенов. Для того, чтобы определить части речи токенов, сначала мы токенизируем переданное нам предложение, затем делаем то, что мы делали при обучении — а именно, перекладываем информацию о символах в тензор, затем, в цикле, применяем нашу модель для каждого предложения. В результате применения нашей модели к каждому предложению мы получаем трёхмерный тензор, первая размерность этого тензора соответствует количеству предложений, вторая — это количество меток, а третья — это максимальная длина предложения. Таким образом, для каждого предложения и для каждого токена у нас есть распределение вероятностей по меткам. И здесь мы просто выбираем для каждого токена наиболее вероятную метку. Затем преобразовываем полученную информацию в формат, удобный для человека, то есть мы преобразовываем номера тэгов в их строковое название

In [24]:
single_token_pos_tagger = POSTagger(single_token_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
sentence_level_pos_tagger = POSTagger(sentence_level_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)

Предложений про "глокую куздру" в обучающей выборке не было. Таким образом это тест на неизвестные слова, а ещё есть несколько тестов на контекст — на учёт контекста, а именно здесь: "ведро дало течь" — это существительное, а "вода стала течь" — это глагол. "Сорок сорок" — здесь одно из слов числительное, а другое — существительное. .

In [25]:
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

Мы видим, что, например, в последнем предложений "сорок сорок", модель назначает просто наиболее вероятный тэг: слово "сорок" всё-таки чаще используются как числительное. Для неизвестных слов модель отработала просто отлично. Хотя допустила пару ошибок, а именно — "штеко" — это не существительное, а наречие. 

In [26]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 137.75it/s]                    

мама-NOUN мыла-VERB раму-NOUN

косил-VERB косой-ADJ косой-ADJ косой-ADJ

глокая-ADJ куздра-NOUN штеко-ADV будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-NOUN дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-PART три-NUM будет-AUX дырка-NOUN

три-NUM да-PART три-NUM будет-AUX шесть-NUM

сорок-NOUN сорок-NOUN



Ну что ж, теперь — вторая модель. Мы сразу видим, что "сорок сорок" уже разобраны более-менее правильно, то есть одно из этих слов — это числительное, а второе — это существительное. Также в предложении про "глокую куздру" исправлена ошибка с наречием, в предложении про "пирожки" эта нейросеть уже смогла различить ситуации, в которых и слово "печь" используется как существительное, и как глагол. Забавно, что во втором аналогичном предложений нейросеть тоже отличила две ситуации, но перепутала. То есть, в том случае, где должен был быть глагол, получилось существительное, а во втором случае должно было быть существительное, а получился глагол. Предложение про "косого косого", который "косил косой" тоже разобрано чуть лучше, хотя и не идеально. Предложения про "три да три будет шесть" или "будет дырка" разобрались так же, предположительно — потому, что мы использовали только три слоя свёрток для учёта контекста токенов и, возможно, нейросети просто не хватило рецептивного поля, то есть для слова "три" она не могла увидеть, что в конце предложения есть слово "шесть". Мы увидели, что, действительно, для задачи определения частей речи, учитывать контекст токенов действительно важно, в определённых случаях. 

In [27]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00, 156.61it/s]                    

мама-NOUN мыла-NOUN раму-NOUN

косил-VERB косой-ADJ косой-ADJ косой-NOUN

глокая-ADJ куздра-NOUN штеко-NOUN будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-NOUN дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-CCONJ три-NUM будет-VERB дырка-NOUN

три-NUM да-CCONJ три-NUM будет-VERB шесть-NUM

сорок-NOUN сорок-NOUN



## Свёрточный модуль своими руками

Давайте копнём чуть чуть глубже — посмотрим, как же можно реализовать свёртки своими руками. Для этого давайте опишем класс — все pytorch-модули наследуются от базового класса "nn.module", давайте реализуем не весь, но основной функционал стандартного модуля "nn.conv1d". При этом, давайте будем, по возможности, делать так, чтобы наш модуль можно было использовать как замену — один в один. Итак, входные параметры — это количество исходных каналов, количество результирующих каналов, размер ядра и размер паддинга. В конструкторе а мы создаём два тензора — это наши веса. "self.weight" — это ядро свёртки. В отличие от стандартного модуля, мы будем использовать здесь прямоугольную матрицу, в этой матрице количество строк равно количеству входных каналов умноженному на размер свёртки (на размер ядра), а количество столбцов этой матрицы равно количеству выходных каналов. Инициализируем этот тензор мы шумом из нормального распределения с маленькой дисперсией. Также у нас есть второй тензор параметров — это смещение. Это просто вектор размерности, равной количеству выходных каналов, его мы инициализируем нулями. Теперь давайте посмотрим на метод "forward". На вход одномерные свёртки принимают трёхмерные тензоры, первая размерность которых соответствует размеру батча, вторая — количеству входных каналов, и третья — длине последовательности. А в качестве результата возвращают также трёхмерный тензор размерности ["количество элементов в батче", "количество выходных каналов" на "новую длину последовательности"]. Она может либо остаться прежней, либо уменьшится, либо увеличиться — это зависит от размеров ядра и от паддинга. Сначала мы делаем паддинг (вот этот кусок кода отвечает за паддинг). Здесь мы реализовали паддинг только нулями. В случае одномерных свёрток, паддинг заключается в том, что мы увеличиваем третье измерение (удлиняем тензор по третьему измерению — то есть, по длине последовательности) на указанное количество элементов, как спереди, так и сзади. Для этого мы создаём тензор нулей, а затем — конкатенируем его вместе с исходным тензором признаков, так, чтобы паддинги были и в начале тензора, и в конце. И нам нужно вычислить заново длину последовательности, переприсвоить переменной. Далее мы подготавливаем матрицу признаков. Как мы это делаем? Вот этот кусок кода отвечает за подготовку признаков. Предположим, что на вход нам пришла вот такая матрица. В ней каждый столбец соответствует какому-то элементу последовательности. Давайте их условно перенумеруем "1, 2, 3, 4, 5". И у нас ядро свёртки, допустим, равно трём. Тогда мы из исходной матрицы сформируем новую матрицу, в которой количество столбцов — новое, оно вычисляется как длина исходной последовательности минус размер ядра плюс 1 (длина исходной последовательности, конечно, берётся уже с учётом паддинга). Для примера — допустим, что у нас нету паддинга. Итак — как мы будем готовить матрицу признаков? Мы будем перебирать все смещения от 0 до размера ядра - 1 и для каждого смещения мы будем брать фрагмент исходной матрицы, начиная с этого смещения, длины равной результирующей длине. Давайте на примере — для исходной последовательности длины 5 и ядра свёртки 3, длина выходной последовательности будет также равна 3 (то есть, chunk size будет равен 3). Тогда — перебирать все смещения от 0 до 2 и выбирать фрагмент исходной матрицы (сначала мы берём такую под-матрицу, затем берём начиная со второго элемента, затем — начиная с третьего элемента). Потом, когда мы перебрали все возможные смещения, мы объединяем все эти кусочки в одну матрицу, причём объединяем её по второму измерению, то есть по измерению признаков — мы получаем, в итоге, вот такую новую матрицу, в ней три столбца, а количество строк равно количеству исходных каналов умноженное на размер ядра свёртки (здесь у нас, по сути, значения будут такие). Таким образом, мы реализовали скользящее окно, которым мы идём по данным. То есть, каждый столбец этой матрицы содержит признаки всех данных, которые попадают в скользящее окно, которым мы идём по исходной последовательности и преобразовываем исходную последовательность в каждом окне.   

12345->123,234,345.  

После объединения кусочков в одну матрицу признаков мы её транспонируем и применяем ядро свёртки. Делать мы это будем с помощью функции "torch.bmm" ("BMM" расшифровывается как "batch matrix multiplication", то есть это пакетное матричное перемножение). эта функция принимает на вход два трёхмерных тензора и, для каждой пары матриц в этих тензорах, делает матричное умножение. Первый тензор — это признаки, которые мы только что составили из кусочков, и второй тензор — это наше ядро свёртки, но наше ядро свёртки — это матрица, а нам нужно получить трёхмерный тензор. Для этого мы добавляем фиктивное нулевое измерение, соответствующее размеру батча и вызываем функцию "expand" ("expand" изменяет размер тензора, но, при этом, она делает это без выделения дополнительной памяти, то есть снаружи выглядит, что тензор большой, а на самом деле это — просто плоская матрица). Итак, ядро свёртки применили, теперь добавляем смещение и транспонируем полученную матрицу так, чтобы у нас смысл измерений в результирующем тензоре соответствовал смыслу измерений во входном тензоре, то есть — сначала размер батча, потом количество каналов, а потом — пространственные измерения, то есть длина последовательности. 

In [28]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                                   requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)
    
    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""

        batch_size, src_channels, sequence_len = x.shape        
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)
            sequence_len = x.shape[-1]

        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

In [29]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

Количество параметров 84882


Попробуем обучить модель для определения частей речи токенов, но свёрточный модуль заменим на наш свёрточный модуль, возьмём за основу модель, которая учитывает контекст токенов, и создадим экземпляр этой модели, и передадим туда все те же самые параметры, которые передавали и раньше, но добавим ещё один — а именно, скажем ей "используй, пожалуйста, наш модуль — не стандартный nn.conv1d из pytorch, а наш модуль, который мы только что описали". Как мы видим, количество параметров в результате мы получили ровно такое же. 

In [30]:
(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                      train_dataset,
                                                      test_dataset,
                                                      F.cross_entropy,
                                                      lr=5e-3,
                                                      epoch_n=10,
                                                      batch_size=64,
                                                      device='cuda',
                                                      early_stopping_patience=5,
                                                      max_batches_per_epoch_train=500,
                                                      max_batches_per_epoch_val=100,
                                                      lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                 factor=0.5,
                                                                                                                                 verbose=True))

Эпоха 0
Эпоха: 501 итераций, 93.38 сек
Среднее значение функции потерь на обучении 0.0749039586499601
Среднее значение функции потерь на валидации 0.018735384135184312
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 93.42 сек
Среднее значение функции потерь на обучении 0.020598441505518383
Среднее значение функции потерь на валидации 0.015261760359574663
Новая лучшая модель!

Эпоха 2
Эпоха: 501 итераций, 93.48 сек
Среднее значение функции потерь на обучении 0.017871170827074203
Среднее значение функции потерь на валидации 0.013712439824375186
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 93.59 сек
Среднее значение функции потерь на обучении 0.016590953531290243
Среднее значение функции потерь на валидации 0.013132767922401723
Новая лучшая модель!

Эпоха 4
Эпоха: 501 итераций, 93.65 сек
Среднее значение функции потерь на обучении 0.015701849050745516
Среднее значение функции потерь на валидации 0.012663036658622251
Новая лучшая модель!

Эпоха 5
Эпоха: 501 итераций, 93.68 сек
Сре

Обучаем нашу модель, используя ту же самую функцию. Любопытно, что модель, которая использует наш свёрточный модуль, на одну эпоху требует 52 секунды. А модель, которая использовала стандартный свёрточный модуль требовала примерно 120 секунд на одну эпоху, то есть получается, что наш модуль чуть-чуть побыстрее. Эта разница имеет значение только для той версии pytorch, которую используем мы сейчас, в другой версии pytorch разница может быть совершенно другая. Наиболее вероятно, что это ускорение вызвано тем, что наша реализация свёрток узкоспециализирована, в ней нету кучи разных вариантов паддинга, в ней нет механизма прореживания ядра, а также нет страйдов — в ней нельзя задавать шаг, с которым нужно идти скользящим окном по исходной последовательности. Другими словами, она проще гораздо, чем стандартная реализация свёрток. Таким образом, иногда имеет смысл что-то реализовать самому, но надо помнить, что в машинном обучении, часто, гораздо важнее быстро проверять разные гипотезы, а для этого лучше использовать стандартные модули, которые проверены, работают надёжно в разных ситуациях, они универсальны и вам не нужно тратить время на написание своих модулей. Свои модули имеет смысл писать только тогда, когда вы точно знаете ту архитектуру, которая вам нужна для решения вашей прикладной задачи, и вы хотите её ускорить. Например — для того, чтобы уметь запускать её на мобильном телефоне.

In [31]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

1526it [00:43, 34.92it/s]                               


Среднее значение функции потерь на обучении 0.009122215211391449
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.93      0.94      0.94     85589
         ADP       1.00      0.99      1.00     81963
         ADV       0.93      0.91      0.92     44101
         AUX       0.92      0.88      0.90      7535
       CCONJ       0.95      0.97      0.96     30432
         DET       0.91      0.94      0.92     21968
        INTJ       0.93      0.35      0.50        78
        NOUN       0.98      0.97      0.97    214497
         NUM       0.95      0.96      0.96     13746
        PART       0.96      0.90      0.93     26638
        PRON       0.95      0.93      0.94     38438
       PROPN       0.92      0.97      0.95     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.90      0.93      0.92     16219
         SYM       1.00      0.99      1.00       840
        VERB    

100%|██████████| 206/205.75 [00:05<00:00, 35.03it/s]


Среднее значение функции потерь на валидации 0.010301370173692703
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.93      0.93      0.93     11222
         ADP       1.00      0.99      1.00     10585
         ADV       0.93      0.91      0.92      6165
         AUX       0.91      0.86      0.88      1108
       CCONJ       0.95      0.97      0.96      4410
         DET       0.90      0.92      0.91      3085
        INTJ       1.00      0.27      0.43        11
        NOUN       0.97      0.96      0.97     27974
         NUM       0.95      0.94      0.95      1829
        PART       0.95      0.90      0.92      3875
        PRON       0.95      0.92      0.93      5598
       PROPN       0.91      0.95      0.93      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.89      0.92      0.91      2258
         SYM       1.00      0.96      0.98        53
        VERB   

Перейдём к оценке качества модели, которая использует наши свёртки. Видим, что она достигает f-меры 0.91, таким образом мы видим, что мы свёртки реализовали правильно, модели работает, всё отлично. Давайте подытожим. В этом семинаре мы рассмотрели, как можно применять свёрточные нейросети для обработки текстов. Мы описали базовую архитектуру — свёрточный ResNet, а также применили её в разных ситуациях для анализа контекста символов и для анализа контекста токенов. Демонстрировали мы работу свёрточной нейросети на задаче предсказания частей речи токенов. Эта задача относится к области лингвистического анализа, то есть анализа структуры текстов. Основная сложность в этой задаче, обычно, заключается в том, чтобы правильно определять часть речи для омонимов, то есть для слов, которые пишутся одинаково, но, на самом деле, имеют разные части речи. Другая сложность заключается в том, чтобы правильно определять части речи для неизвестных слов, то есть каких-нибудь неологизмов или специальной редкой лексики (научной, например). В результате семинара мы увидели, что учёт контекста токенов действительно важен в определённых случаях, хотя, чаще всего, часть речи можно определить, просто посмотрев на само слово. Модели, которые мы обучили в этом семинаре — достаточно неплохие, хотя промышленные POS тэггеры работают с гораздо более высоким качеством (на уровне 0.97 f-мер). Кроме того, мы реализовали свёрточный модуль своими руками и убедились в том, что он работает. 

In [32]:
!gpustat

snwbl                       Wed Feb  9 00:28:16 2022  470.103.01
[0] NVIDIA GeForce RTX 3060 | 47'C,   0 % |  7360 / 12020 MB | dmitry(6903M) gdm(101M) dmitry(315M) dmitry(24M)
